In [6]:
#refine mesh & verify fem error order
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
import math
import numpy as np

mesh = Mesh(unit_square.GenerateMesh(maxh=0.5))

tau = 0.0001
tend = 1
pi = math.pi
u0 = sin(pi*x)*sin(pi*y)

fes = H1(mesh, order=1, dirichlet="bottom|right|left|top")
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform)
a = BilinearForm(aform)
mstar = BilinearForm(mform+tau*aform)

gfu = GridFunction(fes)
gfu.Set(u0)
gfut = GridFunction(gfu.space, multidim=0)
gfut.AddMultiDimComponent(gfu.vec)

t = Parameter(0.0)
uexa = sin(pi*x)*sin(pi*y)*cos(t)
funcf = -sin(pi*x)*sin(pi*y)*sin(t) + 2*pi**2*sin(pi*x)*sin(pi*y)*cos(t)
f = LinearForm(fes)
f += funcf*v*dx

level = 0
Nt = int(tend/tau)

l = []
while level < 5:
    m.Assemble()
    a.Assemble()
    mstar.Assemble()
    mstarinv = mstar.mat.Inverse(fes.FreeDofs())
    
    #TimeStepping()
    for j in range(Nt):
        t.Set((j+1)*tau)
        f.Assemble()
        res = f.vec - a.mat * gfu.vec
        w = mstarinv * res
        gfu.vec.data += tau*w
        gfut.AddMultiDimComponent(gfu.vec)
    err = math.sqrt(Integrate((gfut.MDComponent(Nt-1)-uexa)**2, mesh, order=5))
    print("level = ", level, "ndof = ", fes.ndof, "err = ", err)
    l.append(err)
    #level +=1
    Draw(mesh)
    mesh.Refine()
    level += 1
    
    fes.Update()
    gfu.Update()
    gfu.Set(u0)
    gfut = GridFunction(gfu.space, multidim=0)
    gfut.AddMultiDimComponent(gfu.vec) 



level =  0 ndof =  8 err =  0.7470182699719906


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

level =  1 ndof =  21 err =  0.03896401970397641


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

level =  2 ndof =  65 err =  0.0096227070907864


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

level =  3 ndof =  225 err =  0.0028270936627974048


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

level =  4 ndof =  833 err =  0.0007474463491909417


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [7]:
for j in range(4):
    print(l[j]/l[j+1])

19.17200216115677
4.04917445126059
3.403745414386005
3.7823365728624343


In [65]:
# semi linear parabolic equation
# time method BE for test
# solved by the Picard dynamical iteration
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
import math
import numpy as np
from time import sleep
from ngsolve.solvers import *

# mesh
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

# some constants
tau = 0.01
tend = 100
pi = math.pi
Nt = int(tend/tau)

# fem space
fes = H1(mesh, order=3,dirichlet="bottom|right|left|top")
u,v = fes.TnT()

#time = 0.0
# initial value function
u0 = sin(pi*x)*sin(pi*y)
gfu = GridFunction(fes)
gfu.Set(u0)
#scene = Draw(gfu, deformation=True, autoscale=False, max=1, min=-0.2)
#sleep(2)

#timestepping with nonlinear term 
gfuref = GridFunction(gfu.space, multidim=0) # WR reference solution 
gfuref.AddMultiDimComponent(gfu.vec)
for j in range(Nt):
    nl = BilinearForm(gfu.space)
    nl += (u*v + tau*grad(u)*grad(v) -tau*exp(u/(1+u))*v-gfuref.MDComponent(j)*v)*dx
    Newton(nl,gfu,freedofs=gfu.space.FreeDofs(),maxit=100,maxerr=1e-11,inverse="sparsecholesky",dampfactor=1,printing=False)
    #scene.Redraw()
    #sleep(0.2)
    gfuref.AddMultiDimComponent(gfu.vec)

mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()
mstar = BilinearForm(mform+tau*aform).Assemble()
mstarinv = mstar.mat.Inverse(fes.FreeDofs())

# WR iteration Picard
# initialization
gfu.Set(u0)
gfutold = GridFunction(gfu.space, multidim=0)
for j in range(Nt+1):
    gfutold.AddMultiDimComponent(gfu.vec)

def maxerrnorm(gfutnew,gfuref):
    "max L2 norm iteration error"
    errv = np.zeros(Nt)
    for k in range(Nt):
        errv[k] = math.sqrt(Integrate((gfuref.MDComponent(k+1)-gfutnew.MDComponent(k+1))**2, mesh, order=5))
        if errv[k] >10:
            print("-------------------", k, errv[k])
    return max(errv)
    
# main iteration
wrer = np.zeros(8)
wrer[0] = maxerrnorm(gfutold,gfuref)
print(0, wrer[0])
for it in range(7):
    gfu.Set(u0)
    gfutnew = GridFunction(gfu.space, multidim=0)
    gfutnew.AddMultiDimComponent(gfu.vec)
    for j in range(Nt): 
        # right hand relaxation term
        f = LinearForm(fes)
        preu = gfutold.MDComponent(j+1)
        funcf = exp(preu/(1+preu))
        f += funcf*v*dx
        f.Assemble()
        
        res = f.vec - a.mat * gfu.vec
        w = mstarinv * res
        gfu.vec.data += tau*w
        #Draw(gfu)
        gfutnew.AddMultiDimComponent(gfu.vec)

    gfutold = gfutnew
    wrer[it+1] = maxerrnorm(gfutold,gfuref)
    print(it+1, wrer[it+1])



0 0.4568323622060708
1 0.016287573982251363
2 0.0007679609764973256
3 3.6606650610166255e-05
4 1.7458076501353884e-06
5 8.326087750987435e-08
6 3.970868874482966e-09
7 6.7785784286571e-10


In [63]:
# semi linear parabolic equation
# solved by the Picard parareal dynamical iteration
# time method BE for fine and coarse propagators
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
import math
import numpy as np
from time import sleep
from ngsolve.solvers import *

# mesh
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

# some constants
tau = 0.01
tend = 10
pi = math.pi
Nt = int(tend/tau)

# fem space
fes = H1(mesh, order=3,dirichlet="bottom|right|left|top")
u,v = fes.TnT()

#time = 0.0
# initial value function
u0 = sin(pi*x)*sin(pi*y)
gfu = GridFunction(fes)
gfu.Set(u0)
#scene = Draw(gfu, deformation=True, autoscale=False, max=1, min=-0.2)
#sleep(2)

#timestepping with nonlinear term 
gfuref = GridFunction(gfu.space, multidim=0) # WR reference solution 
gfuref.AddMultiDimComponent(gfu.vec)
for j in range(Nt):
    nl = BilinearForm(gfu.space)
    nl += (u*v + tau*grad(u)*grad(v) -tau*exp(u/(1*(1+u)))*v-gfuref.MDComponent(j)*v)*dx
    Newton(nl,gfu,freedofs=gfu.space.FreeDofs(),maxit=100,maxerr=1e-11,inverse="sparsecholesky",dampfactor=1,printing=False)
    #scene.Redraw()
    #sleep(0.2)
    gfuref.AddMultiDimComponent(gfu.vec)

mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()
mstar = BilinearForm(mform+tau*aform).Assemble()
mstarinv = mstar.mat.Inverse(fes.FreeDofs())

# WR Parareal iteration
# initialization
gfu.Set(u0)
gfuextold = GridFunction(gfu.space, multidim=0)
for j in range(Nt+1):
    gfuextold.AddMultiDimComponent(gfu.vec)

def maxerrnorm(rgfutnew,rgfuref):
    "max L2 norm iteration error"
    errv = np.zeros(Nt)
    for k in range(Nt):
        errv[k] = math.sqrt(Integrate((rgfuref.MDComponent(k+1)-rgfutnew.MDComponent(k+1))**2, mesh, order=5))
        if errv[k] >10:
            print("-----", k, errv[k])
    return max(errv)

# parareal setting
Nf = 25         
Nc = int(Nt/Nf) #number of subintervals
gfutold = GridFunction(gfu.space, multidim=0)
for j in range(Nc+1):
    gfutold.AddMultiDimComponent(gfu.vec)

mstarcoa = BilinearForm(mform + Nf*tau*aform).Assemble()
mstarcoainv = mstarcoa.mat.Inverse(fes.FreeDofs())

def coarse(j,cfu):
    cgfu = GridFunction(fes)
    cgfu.Set(cfu)
    f = LinearForm(fes)
    preu = gfuextold.MDComponent(j*Nf+1)
    funcf = exp(preu/(1*(1+preu)))
    f += funcf*v*dx
    f.Assemble()

    res = f.vec - a.mat * cgfu.vec
    w = mstarcoainv * res
    cgfu.vec.data += Nf*tau*w
    return cgfu.vec

def fine(j,ffu):
    fgfu = GridFunction(fes)
    fgfu.Set(ffu)
    for k in range(Nf):
        f = LinearForm(fes)
        preu = gfuextold.MDComponent(j*Nf+k+1)
        funcf = exp(preu/(1*(1+preu)))
        f += funcf*v*dx
        f.Assemble()
        res = f.vec - a.mat * fgfu.vec
        w = mstarinv * res
        fgfu.vec.data += tau*w
    return fgfu.vec

def extension():
    egfu = GridFunction(fes)
    for j in range(Nc):
        egfu.Set(gfutnew.MDComponent(j))
        for k in range(Nf):
            f = LinearForm(fes)
            preu = gfuextold.MDComponent(j*Nf+k+1)
            funcf = exp(preu/(1*(1+preu)))
            f += funcf*v*dx
            f.Assemble()
            res = f.vec - a.mat * egfu.vec
            w = mstarinv * res
            egfu.vec.data += tau*w
            gfuextnew.AddMultiDimComponent(egfu.vec)

# main iteration
wrpit = 7
wrer = np.zeros(wrpit+1)
wrer[0] = maxerrnorm(gfuextold,gfuref)
print(0, wrer[0])
for it in range(wrpit):
    wrer[it] = maxerrnorm(gfuextold,gfuref)
    gfu.Set(u0)
    gfuextnew = GridFunction(gfu.space, multidim=0)
    gfuextnew.AddMultiDimComponent(gfu.vec)
    for pit in range(1): # inner parareal iteration 
        gfutnew = GridFunction(gfu.space, multidim=0)
        gfutnew.AddMultiDimComponent(gfu.vec)
        for j in range(Nc):
            #tempu = GridFunction(fes)
            tempuvec = fine(j,gfutold.MDComponent(j)) + coarse(j,gfutnew.MDComponent(j)) - coarse(j,gfutold.MDComponent(j))
            gfutnew.AddMultiDimComponent(tempuvec)
            #print(math.sqrt(Integrate((gfutold.MDComponent(j)-gfutnew.MDComponent(j))**2, mesh, order=5)))
        gfutold = gfutnew
        
    extension()          
    gfuextold = gfuextnew  
    wrer[it+1] = maxerrnorm(gfuextold,gfuref)
    print(it+1, wrer[it+1])

0 0.4568323622060708
1 0.05337220770407313
2 0.010759480145660368
3 0.0019903401608000445
4 0.00037477905784742986
5 7.031476435861399e-05
6 1.3201505378969616e-05
7 2.4782428369729976e-06


In [14]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.solvers import *
import scipy

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
fes = H1(mesh, order = 2, dirichlet = "left|right")
u,v = fes.TnT()
a = BilinearForm(fes)
a += (grad(u)*grad(v) + 1/3*u**3*v - 10*v)*dx

gfu = GridFunction(fes)
gfu.Set((x*(1-x))**4*(y*(1-y))**4)
Draw(gfu)
Newton(a,gfu,freedofs=gfu.space.FreeDofs(),maxit=100,maxerr=1e-11,inverse="sparsecholesky",dampfactor=1,printing=True)
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

Newton iteration  0
err =  2.886742631783149
Newton iteration  1
err =  0.1104202770090924
Newton iteration  2
err =  0.00044980563721634664
Newton iteration  3
err =  7.341961621094145e-09
Newton iteration  4
err =  1.1509317217622664e-15


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene